<a href="https://colab.research.google.com/github/Saumibhattacharya/Data-science/blob/master/COVID_DATA_ANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
from bs4 import BeautifulSoup as soup
from datetime import date, datetime
from urllib.request import Request, urlopen
 

In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
import seaborn as sns
import gc
import warnings
warnings.filterwarnings("ignore")

from pandas_profiling import ProfileReport

Web Scrapping


In [ ]:
today = datetime.now()
yesterday_data = "%s %d, %d" %(date.today().strftime('%b'), today.day-1, today.year)
yesterday_data

'Jul 16, 2021'

In [ ]:
url = "https://www.worldometers.info/coronavirus/#countries"
req = Request(url, headers={'User-Agent':"Mozilla/5.0"})

webpage = urlopen(req)
page = soup(webpage, 'html.parser')

#page

In [ ]:
table = page.findAll('table',{'id': 'main_table_countries_yesterday'})

containers = table[0].findAll('tr',{'style': ''})
title = containers[0]

del containers[0]

all_data = []
clean = True

for country in containers:
  country_data = []
  country_container = country.findAll('td')

  if country_container[1].text == 'china':
    continue
  for i in range(1, len(country_container)):
    final_feature = country_container[i].text
    if clean:
      if i!=1 and i!=len(country_container)-1:
        final_feature = final_feature.replace(",","")

        if final_feature.find('+') !=-1:
          final_feature = final_feature.replace("+", "")
          final_feature = float(final_feature)
        elif final_feature.find('-') !=-1:
          final_feature = final_feature_replace('-', '')
          final_feature = float(final_feature)*-1
      if final_feature =='N/A':
        final_feature =0
      elif final_feature =='' or final_feature== " ":
        final_feature = -1
      country_data.append(final_feature)
  
  all_data.append(country_data)


In [ ]:
all_data

In [ ]:
df = pd.DataFrame(all_data)
df.drop([15,16,17,18,19,20], inplace = True,axis =1)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,World,190270157,562565.0,4091489,8653.0,173479901,347758.0,12698767,79204,24410,524.9,-1,-1,-1,All
1,China,92183,36.0,4636,-1.0,87011,6.0,536,8,64,3,160000000,111163,1439323776,Asia
2,USA,34929856,40529.0,624606,293.0,29358531,16618.0,4946719,4876,104888,1876,516064802,1549656,333019037,North America
3,India,31063987,38112.0,413123,560.0,30220183,43877.0,430681,8944,22283,296,440023239,315635,1394086861,Asia
4,Brazil,19308109,45591.0,540500,1450.0,17951699,34510.0,815910,8318,90171,2524,54204686,253141,214128660,South America


In [ ]:
column_labels = ['Country','Total cases', 'New cases', 'Total Deaths', 'New Deaths', 'Total Recovered', 'New recovered', 'Active cases','Serious cases', 'Total cases/1M', 'Total deaths/1M', 'Total tests','Tests/1M', 'Population', 'Continents'] 
df.columns = column_labels
df.head()

,Country,Total cases,New cases,Total Deaths,New Deaths,Total Recovered,New recovered,Active cases,Serious cases,Total cases/1M,Total deaths/1M,Total tests,Tests/1M,Population,Continents
0,World,190270157,562565.0,4091489,8653.0,173479901,347758.0,12698767,79204,24410,524.9,-1,-1,-1,All
1,China,92183,36.0,4636,-1.0,87011,6.0,536,8,64,3,160000000,111163,1439323776,Asia
2,USA,34929856,40529.0,624606,293.0,29358531,16618.0,4946719,4876,104888,1876,516064802,1549656,333019037,North America
3,India,31063987,38112.0,413123,560.0,30220183,43877.0,430681,8944,22283,296,440023239,315635,1394086861,Asia
4,Brazil,19308109,45591.0,540500,1450.0,17951699,34510.0,815910,8318,90171,2524,54204686,253141,214128660,South America


In [ ]:
for label in df.columns:
  if label !='Country' and label != 'Continents':
    df[label] = pd.to_numeric(df[label])

In [ ]:
df['%inc new cases']=df['New cases']/df['Total cases']*100
df['%inc new deaths']=df['New Deaths']/df['Total Deaths']*100
df['%inc new recovered']=df['New recovered']/df['Total Recovered']*100


In [ ]:
df.head()

,Country,Total cases,New cases,Total Deaths,New Deaths,Total Recovered,New recovered,Active cases,Serious cases,Total cases/1M,Total deaths/1M,Total tests,Tests/1M,Population,Continents,%inc new cases,%inc new deaths,%inc new recovered
0,World,190270157,562565.0,4091489,8653.0,173479901,347758.0,12698767,79204,24410.0,524.9,-1,-1,-1,All,0.295666,0.211488,0.200460
1,China,92183,36.0,4636,-1.0,87011,6.0,536,8,64.0,3.0,160000000,111163,1439323776,Asia,0.039053,-0.021570,0.006896
2,USA,34929856,40529.0,624606,293.0,29358531,16618.0,4946719,4876,104888.0,1876.0,516064802,1549656,333019037,North America,0.116030,0.046910,0.056604
3,India,31063987,38112.0,413123,560.0,30220183,43877.0,430681,8944,22283.0,296.0,440023239,315635,1394086861,Asia,0.122689,0.135553,0.145191
4,Brazil,19308109,45591.0,540500,1450.0,17951699,34510.0,815910,8318,90171.0,2524.0,54204686,253141,214128660,South America,0.236124,0.268270,0.192238


Expolatory Data Analysis

In [ ]:
cases = df[['Total Recovered','Active cases','Total Deaths']].loc[0]
cases_df = pd.DataFrame(cases).reset_index()
cases_df.columns = ["Type","Total"]

cases_df["Percentage"] = np.round(100*cases_df['Total']/np.sum(cases_df['Total']),2)
cases_df["Virus type"] = ["COVID-19" for i in range(len(cases_df))]

cases_df

,Type,Total,Percentage,Virus type
0,Total Recovered,173479901,91.18,COVID-19
1,Active cases,12698767,6.67,COVID-19
2,Total Deaths,4091489,2.15,COVID-19


In [ ]:
fig = px.bar(cases_df,x ='Virus type', y = 'Percentage', color = 'Type', hover_data = ["Total"], width = 600)
fig.show()

In [ ]:
cases = df[['New cases','New recovered','New Deaths']].loc[0]
cases_df = pd.DataFrame(cases).reset_index()
cases_df.columns = ["Type","Total"]

cases_df["Percentage"] = np.round(100*cases_df['Total']/np.sum(cases_df['Total']),2)
cases_df["Virus type"] = ["COVID-19" for i in range(len(cases_df))]

cases_df

,Type,Total,Percentage,Virus type
0,New cases,562565.0,61.22,COVID-19
1,New recovered,347758.0,37.84,COVID-19
2,New Deaths,8653.0,0.94,COVID-19


In [ ]:
fig = px.bar(cases_df,x ='Virus type', y = 'Percentage', color = 'Type', hover_data = ["Total"], width = 600)
fig.show()

In [ ]:
per = np.round(df[["%inc new cases","%inc new deaths","%inc new recovered"]].loc[0],2)

per_df = pd.DataFrame(per)
per_df.columns = ['Percentage']
#per_df["Virus type"] = ["COVID-19" for i in range(len(cases_df))]
per_df

,Percentage
%inc new cases,0.30
%inc new deaths,0.21
%inc new recovered,0.20


In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x = per_df.index, y = per_df['Percentage'], marker_color = ["Yellow", 'blue','red']))
fig.show() 

Continents

In [ ]:
continents_df = df.groupby("Continents").sum().drop("All")  #groupby - continents and sum the numerics and drop rest(String type)
continents_df = continents_df.reset_index()  #indexing
continents_df.head()

,Continents,Total cases,New cases,Total Deaths,New Deaths,Total Recovered,New recovered,Active cases,Serious cases,Total cases/1M,Total deaths/1M,Total tests,Tests/1M,Population,%inc new cases,%inc new deaths,%inc new recovered
0,Africa,6209831,41670.0,156890,976.0,5390740,43695.0,662201,4433,694255.0,10014.0,56526817,5314700,1373781811,17.060620,-108.809264,5.661119
1,Asia,58632642,210027.0,836831,3127.0,55415883,148698.0,2379927,29267,1588655.0,15515.5,1044459835,39097752,4644711537,32.638501,22.669049,22.844297
2,Australia/Oceania,86280,1555.0,1340,1.0,71461,477.0,13478,48,88599.0,658.0,25356755,1865549,41405631,9.236725,102.291093,13.112976
3,Europe,49570053,150441.0,1118046,965.0,46089331,42883.0,2362676,6293,3642030.0,69648.0,1025060867,90418363,748099095,14.151058,-115.026467,3.244720
4,North America,41316676,68580.0,928168,746.0,34659426,34423.0,5729079,11850,1388678.0,19512.0,580636047,31758757,593781775,30.866490,-17.492855,25.538255


In [ ]:
def continent_visulaization(v_list):
  for label in v_list:
    c_df = continents_df[['Continents',label]]
    c_df['Percentage'] = np.round(100*c_df[label]/np.sum(c_df[label]),2)
    c_df['Virus type'] = ["COVID-19" for i in range(len(c_df))]

    fig = px.bar(c_df,x ='Virus type', y = 'Percentage', color = 'Continents', hover_data = [label], width = 600)
    fig.update_layout(title = {"text" : label})
    fig.show()

In [ ]:
cases_list = ['Total cases', 'Active cases','New cases','Serious cases','Total cases/1M']

deaths_list = ['Total Deaths','New Deaths','Total deaths/1M']

recovered_list = ['Total Recovered', 'New recovered', '%inc new recovered']

In [ ]:
continent_visulaization(cases_list)
#continent_visulaization(deaths_list)
#continent_visulaization(recovered_list)

Countries

In [ ]:
#df = df.drop([len(df)-1])
country_df = df.drop([0])

country_df.head()

,Country,Total cases,New cases,Total Deaths,New Deaths,Total Recovered,New recovered,Active cases,Serious cases,Total cases/1M,Total deaths/1M,Total tests,Tests/1M,Population,Continents,%inc new cases,%inc new deaths,%inc new recovered
1,China,92183,36.0,4636,-1.0,87011,6.0,536,8,64.0,3.0,160000000,111163,1439323776,Asia,0.039053,-0.021570,0.006896
2,USA,34929856,40529.0,624606,293.0,29358531,16618.0,4946719,4876,104888.0,1876.0,516064802,1549656,333019037,North America,0.116030,0.046910,0.056604
3,India,31063987,38112.0,413123,560.0,30220183,43877.0,430681,8944,22283.0,296.0,440023239,315635,1394086861,Asia,0.122689,0.135553,0.145191
4,Brazil,19308109,45591.0,540500,1450.0,17951699,34510.0,815910,8318,90171.0,2524.0,54204686,253141,214128660,South America,0.236124,0.268270,0.192238
5,Russia,5907999,25704.0,146868,799.0,5300908,21932.0,460223,2300,40466.0,1006.0,157300000,1077401,145999538,Europe,0.435071,0.544026,0.413740


In [ ]:
country = country_df.columns[1:14]
look_At = 5
fig = go.Figure() #creating fig obj
c = 0
for i in country_df.index :
  if c < look_At:
    fig.add_trace(go.Bar(name = country_df['Country'][i], x = country, y = country_df.loc[i][1:14]))
  else:
    break
  c+=1

fig.update_layout(title = {'text' : f'Top {look_At} countries affected'}, yaxis_type = 'log')   # we see if we only take numeric data, then high value dominates the other. therefore we will take log values.(normalize)
fig.show()